# Biopython - Esercizio4

[MAFFT](https://www.ebi.ac.uk/Tools/msa/mafft/) è un tool di allineamento multiplo sviluppato da EMBL-EBI (European Bioinformatics Institute - European Molecular Biology Laboratory) per sequenze di DNA.

Usare MAFFT (scegliendo ClustalW come formato di output) per allineare i 14 genomi completi di SARS-CoV-2 presenti nel file `covid-sequences.fasta` sequenziati nel novembre 2021 e scaricati dal sito di [NCBI](https://www.ncbi.nlm.nih.gov/sars-cov-2/). Il primo, con identificatore `NC_045512.2`, è il genoma di riferimento.

Trovare tutte le variazioni rispetto al genoma di riferimento.

---

**Variazione**: posizione della colonna di allineamento in cui esiste almeno un genoma che ha mismatch con quello di riferimento.

Esempio di allineamento con variazioni in posizione 8 e 13:

    REF   AAGCTGATTGCACGC-T
    G1    --GCAGAGTGCAGGCCT
    G2    --GCCGAGTGCACGCCT

**Variazione 5**: `T` nel reference e `A` in G1 e `C` in G2.

**Variazione 8**: `T` nel reference e `G` sia in G1 e G2.

**Variazione 13**: `C` nel reference e `G` in G1.

**Variazione 16**: `-` nel reference e `C` sia in G1 che in G2.

---

Si richiede di:
- costruire il data frame delle variazioni in cui le colonne sono tutte le posizioni 1-based delle variazioni e le righe sono indicizzate con l'identificatore del genoma.
- estrarre il genoma con più variazioni e quello con meno variazioni
- ottenere il data frame delle variazioni "complete", cioè in cui tutti i genomi variano rispetto al riferimento.
- produrre il data frame delle variazioni "stabili" in cui tutti i genomi variano allo stesso modo rispetto al riferimento. 
- ottenere la lista delle posizioni in cui c'è un gap nel genoma di riferimento.
- ottenere la lista delle posizioni in cui c'è un gap in almeno uno dei genomi (diversi dal riferimento)

Installare il package `Bio` di Biopython.

Importare il package `Bio`.

In [29]:
import Bio

Importare il package `AlignIO` che è il package per manipolare file contenenti allineamenti multipli in diversi formati (tra cui `clustal` che è quello del file di input).

In [30]:
from Bio import AlignIO

#### Leggere l'allineamento in input

Il package `AlignIO` mette a disposizione la funzione `read` per leggete un allineamento:

       AligIO.read(input_file_name, format)
       
e restituisce un oggetto di tipo `MultipleSeqAlignment` che è un oggetto iterabile contenente oggetti `SeqRecord`, uno per ognuna delle righe dell'allineamento letto.

In [31]:
alignment = AlignIO.read("mafft-alignments.clustalw", "clustal")

La lunghezza dell'allineamento in input (numero di colonne della matrice di allineamento) è:

In [32]:
alignment.get_alignment_length()

29903

Trasformare l'oggetto in una lista di oggetti `SeqRecord`.

[SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet()), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('---------------------AGGTAACAAACCNACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet()), id='OL700521.1', name='<unknown name>', description='OL700521.1', dbxrefs=[]),
 SeqRecord(seq=Seq('---------------CTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet()), id='OL700526.1', name='<unknown name>', description='OL700526.1', dbxrefs=[]),
 SeqRecord(seq=Seq('----------------------------------------------TCTCTTGT...AAA', SingleLetterAlphabet()), id='OL700531.1', name='<unknown name>', description='OL700531.1', dbxrefs=[]),
 SeqRecord(seq=Seq('----------------------------------------------TCTCTTGT...---', SingleLetterAlphabet()), id='OL700532.1', name='<unknown name>', description='OL700532.1', dbxrefs=[]),
 SeqRecord(seq=Seq('-------------------------------------------

#### Eliminare i gap iniziali.

Trovare il più lungo prefisso di soli simboli `-` delle righe dell'allineamento. Supponendo che tale prefisso sia lungo `g`, eliminare da ogni riga dell'allinemento il prefisso di lunghezza `g`.

Ad esempio il seguente allineamento composto da tre righe:

    GTATGTGTCATGTTTTTGCTA
    --ATGTGTCATG-TTT-----
    ----GTGTCATGTTTTTG---
    
presenta un più lungo prefisso di soli simboli `-` di lunghezza `g=4` (terza riga). Eliminando da tutte le righe un prefisso di lunghezza 4 si ottiene:

        GTGTCATGTTTTTGCTA
        GTGTCATG-TTT-----
        GTGTCATGTTTTTG---

In [36]:
alignment

[SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA', SingleLetterAlphabet()), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA', SingleLetterAlphabet()), id='OL700521.1', name='<unknown name>', description='OL700521.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA', SingleLetterAlphabet()), id='OL700526.1', name='<unknown name>', description='OL700526.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA', SingleLetterAlphabet()), id='OL700531.1', name='<unknown name>', description='OL700531.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...---', SingleLetterAlphabet()), id='OL700532.1', name='<unknown name>', description='OL700532.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGT

#### Eliminare i gap finali.

Trovare il più lungo suffisso di soli simboli `-` delle righe dell'allineamento. Supponendo che tale suffisso sia lungo `g`, eliminare da ogni riga il suffisso di lunghezza `g`.

Ad esempio il seguente allineamento composto da tre righe:

        GTGTCATGTTTTTGCTA
        GTGTCATG-TTT-----
        GTGTCATGTTTTTG---
        
presenta un più lungo suffisso di soli simboli `-` di lunghezza `g=5` (seconda riga). Eliminando da tutte le righe un suffisso di lunghezza 5 si ottiene:

        GTGTCATGTTTT
        GTGTCATG-TTT
        GTGTCATGTTTT

In [38]:
alignment

[SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA', SingleLetterAlphabet()), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA', SingleLetterAlphabet()), id='OL700521.1', name='<unknown name>', description='OL700521.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA', SingleLetterAlphabet()), id='OL700526.1', name='<unknown name>', description='OL700526.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA', SingleLetterAlphabet()), id='OL700531.1', name='<unknown name>', description='OL700531.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA', SingleLetterAlphabet()), id='OL700532.1', name='<unknown name>', description='OL700532.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGT

#### Creare la lista degli identificatori dei genomi

In [39]:
index_list = [row.id for row in alignment]

In [12]:
index_list

['NC_045512.2',
 'OL700521.1',
 'OL700526.1',
 'OL700531.1',
 'OL700532.1',
 'OL700537.1',
 'OL700524.1',
 'OL700530.1',
 'OL700538.1',
 'OL700543.1',
 'OL700544.1',
 'OL700541.1',
 'OL700533.1',
 'OL700545.1']

#### Creare il dizionario contenente i dati per costruire il data frame

- `key`: posizione 1-based della variazione (posizione della colonna nell'allineamento in input)

- `value`: lista dei simboli allineati coinvolti nella variazione (il primo simbolo deve essere quello del reference, mentre se un genoma non presenta una differenza con il reference si deve inserire la stringa vuota)

In [41]:
df_data

{'186': ['C', '', '', '', '', 'T', '', '', '', '', '', '', '', ''],
 '210': ['G', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T'],
 '241': ['C', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T'],
 '1048': ['G', '', '', 'T', '', '', '', '', '', '', '', '', '', ''],
 '1244': ['G', '', '', '', '', '', '', '', '', '', 'A', '', '', ''],
 '1371': ['A', '', '', '', '', '', '', '', '', '', '', 'G', '', ''],
 '1616': ['C', '', '', '', '', '', '', '', '', '', '', '', 'A', ''],
 '1684': ['C', '', '', '', '', '', '', '', '', '', '', '', 'T', 'T'],
 '1843': ['G', '', '', '', '', '', '', '', '', '', '', 'T', '', ''],
 '1889': ['C', '', '', '', '', '', '', '', '', 'T', '', '', '', ''],
 '2462': ['C', '', '', '', 'T', '', '', '', '', '', '', '', '', ''],
 '2929': ['A', '', '', '', '', '', '', '', '', '', '', 'G', '', ''],
 '3037': ['C',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T',
  'T'],
 '3096': ['C', '', '', '', '', '', '', '', 

#### Creare il data frame

    df = pd.DataFrame(data, index = index_list)

In [42]:
df

,186,210,241,1048,1244,1371,1616,1684,1843,1889,...,29095,29119,29402,29409,29509,29543,29648,29700,29742,29781
NC_045512.2,C,G,C,G,G,A,C,C,G,C,...,C,C,G,C,C,G,G,A,G,G
OL700521.1,,T,T,,,,,,,,...,,,T,,,,,,T,
OL700526.1,,T,T,,,,,,,,...,,,T,,,,,,T,
OL700531.1,,T,T,T,,,,,,,...,,,T,,,,,,T,
OL700532.1,,T,T,,,,,,,,...,,,T,,T,T,,,T,
OL700537.1,T,T,T,,,,,,,,...,,,T,,,,,,T,
OL700524.1,,T,T,,,,,,,,...,,,T,,T,,T,,T,
OL700530.1,,T,T,,,,,,,,...,,,T,,,,,,T,
OL700538.1,,T,T,,,,,,,,...,T,,T,,,,,,T,
OL700543.1,,T,T,,,,,,,T,...,,T,T,,,,,,T,T


#### Estrarre il genoma con più variazioni e quello con meno variazioni

Determinare la lista del numero di variazioni per genoma (per tutti i genomi tranne quello di riferimento).

In [45]:
variants_per_genome

[36, 40, 41, 45, 45, 45, 42, 47, 45, 41, 42, 40, 41]

'OL700538.1'

'OL700521.1'

#### Determinare il data frame delle variazioni "complete"

Selezionare dal data frame precedente le sole colonne relative a variazioni "complete".

In [52]:
df_complete

,210,241,3037,14408,15451,16466,21618,21987,22917,22995,...,23604,24410,25469,26767,27638,27752,28461,28881,29402,29742
NC_045512.2,G,C,C,C,G,C,C,G,T,C,...,C,G,C,T,T,C,A,G,G,G
OL700521.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700526.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700531.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700532.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700537.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700524.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700530.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700538.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700543.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T


#### Determinare il data frame delle variazioni "stabili"

Selezionare dal data frame precedente le sole colonne relative a variazioni "stabili".

In [54]:
df_stable

,210,241,3037,14408,15451,16466,21618,21987,22917,22995,...,23604,24410,25469,26767,27638,27752,28461,28881,29402,29742
NC_045512.2,G,C,C,C,G,C,C,G,T,C,...,C,G,C,T,T,C,A,G,G,G
OL700521.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700526.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700531.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700532.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700537.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700524.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700530.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700538.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T
OL700543.1,T,T,T,T,A,T,G,A,G,A,...,G,A,T,C,C,T,G,T,T,T


#### Ottenere la lista delle posizioni in cui c'è un gap nel genoma di riferimento.

In [56]:
ref_gaps

[]

#### Ottenere la lista delle posizioni in cui c'è un gap in almeno uno dei genomi (diversi dal riferimento).

In [58]:
other_gaps

[]